In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
import subprocess

# Function to block IP (Linux-based firewall command example)
def block_ip(ip_address):
    try:
        subprocess.run(["sudo", "iptables", "-A", "INPUT", "-s", ip_address, "-j", "DROP"], check=True)
        print(f"Blocked IP: {ip_address}")
    except Exception as e:
        print(f"Error blocking IP {ip_address}: {e}")

# Function to notify admin
def notify_admin(ip_address, attack_type):
    print(f"Alert: Potential {attack_type} attack detected from {ip_address}. Admin notified.")

# Load dataset (same as IDS)
def loadDataset():
    filename = 'https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/pcap_data.csv'
    trainfile = pd.read_csv(filename)
    data = pd.DataFrame(trainfile).to_numpy()
    label = data[:, 67].astype('str')

    inx_sel = -1 + np.array([38, 47, 37, 48, 11, 9, 7, 52, 10, 36, 1, 34, 4, 17, 19, 57, 21,
                             18, 22, 24, 32, 50, 23, 55, 51, 5, 3, 39, 40, 43, 58, 12, 25,
                             20, 2, 35, 67, 33, 6, 53])

    data = data[:, inx_sel]

    label_map = {"WebDDoS": 0, "BENIGN": 1, "UDP-lag": 2, "DrDoS_NTP": 3, "Syn": 4,
                 "DrDoS_SSDP": 5, "DrDoS_UDP": 6, "DrDoS_NetBIOS": 7, "DrDoS_MSSQL": 8,
                 "DrDoS_SNMP": 9, "TFTP": 10, "DrDoS_DNS": 11}

    label = np.array([label_map.get(l, -1) for l in label])
    filtered_indices = label != -1
    data, label = data[filtered_indices], label[filtered_indices]

    train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.20, stratify=label)
    train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size=0.125, stratify=train_label)

    return train_data.astype('float32'), train_label.astype('int32'), val_data.astype('float32'), val_label.astype('int32'), test_data.astype('float32'), test_label.astype('int32')

# Load data
train_data, train_labelp, val_data, val_labelp, test_data, test_labelp = loadDataset()
train_label = to_categorical(train_labelp, 12)
val_label = to_categorical(val_labelp, 12)
test_label = to_categorical(test_labelp, 12)

# Train model (same as IDS, assumed models_ddos module exists)
inshape = train_data.shape[1]
model = models_ddos.model_conv1D(lr=1e-4, N=64, inshape=inshape)
model.fit(train_data, train_label, epochs=100, batch_size=256, validation_data=(val_data, val_label))

# Predict on test data
pred = model.predict(test_data)
pred_y = pred.argmax(axis=-1)
cm = confusion_matrix(test_labelp, pred_y)

# Implement IPS reactions
attack_classes = {0: "WebDDoS", 2: "UDP-lag", 3: "DrDoS_NTP", 4: "Syn", 5: "DrDoS_SSDP"}
malicious_ips = []
for i, label in enumerate(test_labelp):
    if label in attack_classes:
        ip_address = test_data[i][0]  # Assume first column holds IP (modify if needed)
        malicious_ips.append(ip_address)
        block_ip(ip_address)
        notify_admin(ip_address, attack_classes[label])


IndexError: index 67 is out of bounds for axis 1 with size 26